In [1]:
# 시스템
import os
import sys
import re
# 크롤링
import urllib.request
import requests
import json
from scrapy.http import TextResponse
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm

import pandas as pd

In [2]:
# 태그 데이터를 삭제하는 코드
def tag_remove(word):

    word = re.sub('(<([^>]+)>)', '', word)

    return word

# 한글만 남기는 코드
def preprocess_sentence_kr(w):
    w = w.strip()
    w = re.sub(r"[^0-9가-힣?.!,¿]+", " ", w) # \n도 공백으로 대체해줌
    w = w.strip()
    return w

In [3]:
def blog_content_parsing(url):
    blog_req = requests.get(url)
    blog_bs = BeautifulSoup(blog_req.text, "html.parser")
    
    result = []
    
    for i in blog_bs.select(".se-text"):
        for j in i.select(".se-module-text > p"):
            txt = j.text
            if txt != "\u200b":
                result.append(preprocess_sentence_kr(txt))  
                
    return " ".join(result)

In [4]:
# blog가 네이버 블로그인지 아닌지 판별하는 코드

def naver_blog_check(code):
    
    if code[0].isdigit():
        return False
    else:
        return True
    

In [5]:
# 글자 수 중 반 이상이 title에 들어가있다면 패스 아니면 해당 내용은 다른 내용이라고 판단한다.

def naver_title_check(search_word, title):
    total_cnt = len(title.split())
    cnt = 0
    
    for word in search_word.split():
        
        if word in title:
            cnt += 1
            
    if total_cnt// 2 <= cnt:
        return False
    else:
        return True 
        
        

In [13]:
# 검색 결과가 title에 들어가있다면 해당 내용을 없앤다.
# 아니면 해당 내용은 다른 내용이라고 판단한다.

# 띄어쓰기를 안하고 -로 나눈경우
# 

def naver_title_check(search_word, title):
    # -로 제품이 안나뉜 경우 -> 소문자로만 판단
     
    total_cnt = len(search_word.split())
    cnt = 0
    
    for word in search_word.split():
        word = word.lower()
        if word in title.lower(): # 대소문자 구분 x
            cnt += 1
    if total_cnt == cnt:
        return False
    else:
        return True 

In [27]:
len("sdf asdfs sdfefv dfs")

20

In [30]:
asdf = "MLKFMKALSF"

In [31]:
asdf.lower()

'mlkfmkalsf'

In [37]:
# 모든 단어를 하나씩 나누어서 확인하자.
# 

def naver_title_check(search_word, title):
     
    search_word = preprocess_sentence_kr(search_word)
    
    for words in search_word.split():
        for word in words:
            if word.lower() not in title.lower():
                return True
            


In [24]:
"wf-wfs".split()[0].lower()

'wf-wfs'

In [25]:
if "WF-WEFS".lower()

'wf-wefs'

In [21]:
len("wf-wfs".split())

1

In [19]:
if naver_title_check("wf-wfs", "WF-WEFS"):
    print("yes")
else:
    print("no")

yes


In [20]:
# 이름이 들어오면 해당 내용을 이용하여 블로그를 파싱한다.

# search_word : 어떤 제품을 검색하는지
# repeat_num : 해당 제품이 제목에서 몇번 안나올때까지 검색을 진행할 지
def item_parsing(search_word, start, display, repeat_num):

    
    blog_search = naver_api(search_word, start, display)
    print("start ++++++++++++++++++++")
    
    # result : url
    # writer : 작성자 이름
    # code : 블로그 코드
    # title : 블로그 제목
    # description : 블로그 요약 설명
    blog_result = []
    blog_writer = []
    blog_code = []
    blog_title = []
    blog_description = []
    
    repeat = True
    cnt = 0
    # 제목에서 해당 제품의 이름이 5번 연속 안나온다면 패스
    while repeat:
        
        if cnt == 3:
            break
        
        for blog in blog_search["items"]:
            
             # 블로그 코드
            code = blog["link"][blog["link"].find("logNo=") + 6:]
            if naver_blog_check(code): # 네이버 블로그가 아닌경우 다음으로 넘기기
                continue
            
            
            # 블로그 코드 추가
            blog_code.append(code)
            
            # 이름 추가
            
#             print(f"cnt = {cnt}")
            cnt += 1
            # 이름
            writer = blog["bloggerlink"][23:]
            blog_writer.append(writer)


            # 제목
            title = tag_remove(blog["title"])
            blog_title.append(title)
            
#             if search_word not in title:
#                 cnt += 1
#             else:
#                 cnt = 0
                
#             if cnt == repeat_num:
#                 repeat = False

            # 블로그 요약
            description = tag_remove(blog["description"])
            blog_description.append(description)
            
            blog_result.append(f"https://blog.naver.com/PostView.naver?blogId={writer}&logNo={code}")
        
        if start + display <= blog_search["total"]:
            start += display
            blog_search = naver_api(search_word, start, display)
            print(f"start = {start}")
            print("repeat start ++++++++++++++++++++")
        else:
            print("repeat End")
            repeat = False
            
        cnt += 1
        
    return blog_result, blog_writer, blog_code, blog_title, blog_description
        
    

In [33]:
# 이름이 들어오면 해당 내용을 이용하여 블로그를 파싱한다.

# search_word : 어떤 제품을 검색하는지
# repeat_num : 해당 제품이 제목에서 몇번 안나올때까지 검색을 진행할 지
def item_parsing(search_word, start, display, repeat_num):

    
    blog_search = naver_api(search_word, start, display)
    print("start ++++++++++++++++++++")
    
    # result : url
    # writer : 작성자 이름
    # code : 블로그 코드
    # title : 블로그 제목
    # description : 블로그 요약 설명
    blog_result = []
    blog_writer = []
    blog_code = []
    blog_title = []
    blog_description = []
    
    repeat = True
    cnt = 0
    # 제목에서 해당 제품의 이름이 5번 연속 안나온다면 패스
    while repeat:
        
#         if cnt == 3:
#             break
        
        for blog in blog_search["items"]:
            
             # 블로그 코드
            code = blog["link"][blog["link"].find("logNo=") + 6:]
            if naver_blog_check(code): # 네이버 블로그가 아닌경우 다음으로 넘기기
                continue
                
            
            
            
            

            
            # 이름
            writer = blog["bloggerlink"][23:]
            

            # 제목
            title = tag_remove(blog["title"])
            
            if naver_title_check(search_word, title):
                continue
            
            
#             if search_word not in title:
#                 cnt += 1
#             else:
#                 cnt = 0
                
#             if cnt == repeat_num:
#                 repeat = False

            # 블로그 요약
            description = tag_remove(blog["description"])
            
            
            ## 데이터 추가하는 곳
            
            # 블로그 코드 추가
            blog_code.append(code)
            
            # 이름 추가
            blog_writer.append(writer)
            
            # 제목 추가
            blog_title.append(title)
            
            # 블로그 요약 추가
            blog_description.append(description)
            
            # 본문 크롤링을 위한 URL주소 추가
            blog_result.append(f"https://blog.naver.com/PostView.naver?blogId={writer}&logNo={code}")            
            
        
        if start + display <= blog_search["total"]:
            start += display
            blog_search = naver_api(search_word, start, display)
            print(f"start = {start}")
            print("repeat start ++++++++++++++++++++")
        else:
            print("repeat End")
            repeat = False
            
        
        
    return blog_result, blog_writer, blog_code, blog_title, blog_description
        
    

In [8]:
def naver_api(search_word, start, display):
    client_id = "GvNa2sBgFDA6v7ujnaz0"
    client_secret = "Yo0jOskXlZ"
    
    encText = urllib.parse.quote(search_word)
    
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText # json 결과
    url += f"&start={start}"
    url += f"&display={display}"

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
    else:
        print("Error Code:" + rescode)
    return json.loads(response_body.decode('utf-8'))

In [12]:
t = naver_api("wf-1000xm4 후기", 1, 30)
t

{'lastBuildDate': 'Mon, 18 Apr 2022 13:44:00 +0900',
 'total': 354,
 'start': 1,
 'display': 30,
 'items': [{'title': '무선 이어폰 종결 끝판왕 소니 <b>WF-1000XM4</b> 샀습니다',
   'link': 'https://blog.naver.com/khjkhj2804?Redirect=Log&logNo=222645074782',
   'description': '<b>WF-1000XM4</b> 가 현재 끝판왕인 이유 일단, 가격 깡패 &amp; 음질 깡패 최고임 이제 한 달 넘게 사용하고 쓰는 <b>후기</b> 무엇보다 고성능 라인업들 가격대비 가격 앞 자리 단위 자체가 내려가 있습니다. 가격 자체가 깡패인데... ',
   'bloggername': '겜잘알 갓플머',
   'bloggerlink': 'https://blog.naver.com/khjkhj2804',
   'postdate': '20220211'},
  {'title': '소니 <b>WF-1000XM4</b> 노이즈 캔슬링 이어폰 개봉 <b>후기</b>... ',
   'link': 'https://blog.naver.com/rladnjsxo64?Redirect=Log&logNo=222571874963',
   'description': '지난 몇 주간 소니 <b>WF-1000XM4</b>에 대한 사용 <b>후기</b>를 남겼는데 개봉기를 빼놓을 수 없죠! 오늘은 조금(?) 늦은 개봉기를 작성해보겠습니다. <b>WF-1000XM4</b> 이어폰은 블랙과 실버 2가지 색상으로 출시를 했으며, 제가... ',
   'bloggername': '무라키의 데이 에세이',
   'bloggerlink': 'https://blog.naver.com/rladnjsxo64',
   'postdate': '20211118'},
  {'title': '무선 노이즈 캔슬링 이어폰 소니 <b>WF-1000XM4</b

In [34]:
test = item_parsing("wf-1000xm4 후기", 1, 100, 3)

start ++++++++++++++++++++
start = 101
repeat start ++++++++++++++++++++
start = 201
repeat start ++++++++++++++++++++
start = 301
repeat start ++++++++++++++++++++
repeat End


In [35]:
len(test[1])

74

In [36]:
test[3]

['소니 WF-1000XM4 노이즈 캔슬링 이어폰 개봉 후기... ',
 '무선 노이즈 캔슬링 이어폰 소니 WF-1000XM4 간단 후기',
 '준하는 노이즈캔슬링 소니 WF-1000XM4 무선이어폰 후기',
 '소니 무선 노이즈캔슬링 이어폰 WF-1000XM4 내돈내산 리뷰 후기',
 '[내돈내산 후기] 음질, 노캔 최강 소니 WF-1000XM4',
 '소니 이폰 WF-1000XM4 개봉기와 3개월 사용후기! 유일한... ',
 '소니 무선 이어폰 (SONY WF-1000XM4) 후기',
 'SONY/소니 이어폰) WF-1000XM4 BME 사용후기',
 'SONY WF-1000XM4 후기',
 '등 야외에서 사용해본 WF-1000XM4 노이즈 캔슬링 이어폰 후기',
 '소니 WF-1000XM4 무선이어폰 운동용으로 사용해본 후기',
 '소니 WF-1000xm4, 2주간의 실 사용후기',
 '노이즈캔슬링 소니 이어폰 WF-1000XM4 후기',
 '헤드폰에 도전하다 SONY WF-1000XM4 이어폰 구입 간단 후기',
 '소니 wf-1000xm4 음질 선물과 6주간 사용 후기',
 '성능 SONY WF-1000XM4 블루투스 무선이어폰 리얼 후기',
 '[블루투스 이어폰] 소니 WF-1000XM4 사용 후기ㅣ30대 남자... ',
 '(SONY) WF-1000XM4 노이즈캔슬링이어폰 일주일 사용기 후기',
 '소니 WF-1000XM4 일상 속 사용후기 : 노이즈캔스링이 필요한... ',
 '소니 WF-1000XM4 이어폰 내돈내산 후기 (노이즈 캔슬링... ',
 '소니 WF-1000XM4 사용 후기 (내돈내산)',
 '[Sony] WF-1000XM4 (무선이어폰) 구매 후기',
 '노이즈캔슬링 블루투스 무선 이어폰 소니 WF-1000XM4 사용 후기',
 '소니 WF-1000XM4, 네 번째 이야기: 영화 감상과 케이스 후기',
 '소니 WF-1000XM4 에코 케이스 카세트 디자인 사용후기',
 '소니 WF-1000XM4 구매 및 사용후기 작성해봅니다.

In [38]:
test = item_parsing("wf-1000xm4 후기", 1, 100, 3)

start ++++++++++++++++++++
start = 101
repeat start ++++++++++++++++++++
start = 201
repeat start ++++++++++++++++++++
start = 301
repeat start ++++++++++++++++++++
repeat End


In [40]:
test[3]

['소니 WF-1000XM4 노이즈 캔슬링 이어폰 개봉 후기... ',
 '무선 노이즈 캔슬링 이어폰 소니 WF-1000XM4 간단 후기',
 '준하는 노이즈캔슬링 소니 WF-1000XM4 무선이어폰 후기',
 '소니 무선 노이즈캔슬링 이어폰 WF-1000XM4 내돈내산 리뷰 후기',
 '[내돈내산 후기] 음질, 노캔 최강 소니 WF-1000XM4',
 '소니 이폰 WF-1000XM4 개봉기와 3개월 사용후기! 유일한... ',
 '소니 무선 이어폰 (SONY WF-1000XM4) 후기',
 'SONY/소니 이어폰) WF-1000XM4 BME 사용후기',
 'SONY WF-1000XM4 후기',
 '등 야외에서 사용해본 WF-1000XM4 노이즈 캔슬링 이어폰 후기',
 '소니 WF-1000XM4 무선이어폰 운동용으로 사용해본 후기',
 '소니 WF-1000xm4, 2주간의 실 사용후기',
 '노이즈캔슬링 소니 이어폰 WF-1000XM4 후기',
 '헤드폰에 도전하다 SONY WF-1000XM4 이어폰 구입 간단 후기',
 '소니 wf-1000xm4 음질 선물과 6주간 사용 후기',
 '성능 SONY WF-1000XM4 블루투스 무선이어폰 리얼 후기',
 '[블루투스 이어폰] 소니 WF-1000XM4 사용 후기ㅣ30대 남자... ',
 '(SONY) WF-1000XM4 노이즈캔슬링이어폰 일주일 사용기 후기',
 '소니 WF-1000XM4 일상 속 사용후기 : 노이즈캔스링이 필요한... ',
 '소니 WF-1000XM4 이어폰 내돈내산 후기 (노이즈 캔슬링... ',
 '소니 서비스센터 신용산점 방문 후기[wf 1000xm4 수리]',
 '소니 WF-1000XM4 사용 후기 (내돈내산)',
 '[Sony] WF-1000XM4 (무선이어폰) 구매 후기',
 '노이즈캔슬링 블루투스 무선 이어폰 소니 WF-1000XM4 사용 후기',
 '소니 WF-1000XM4, 네 번째 이야기: 영화 감상과 케이스 후기',
 '소니 WF-1000XM4 에코 케이스 카세트 디자인 사

In [39]:
len(test[1])

96

In [48]:
len(test)

5

In [9]:
search_word = input("검색하고 싶은 단어를 입력해주세요.")

검색하고 싶은 단어를 입력해주세요.랩씨 마카롱 스마트 커버 후기


In [41]:
a,b,c,d,e = test

In [42]:
a

['https://blog.naver.com/PostView.naver?blogId=rladnjsxo64&logNo=222571874963',
 'https://blog.naver.com/PostView.naver?blogId=feena74&logNo=222423856964',
 'https://blog.naver.com/PostView.naver?blogId=supapa13&logNo=222498158979',
 'https://blog.naver.com/PostView.naver?blogId=lifesdh&logNo=222414669555',
 'https://blog.naver.com/PostView.naver?blogId=xerotype2&logNo=222415306975',
 'https://blog.naver.com/PostView.naver?blogId=dongchimi212&logNo=222652612911',
 'https://blog.naver.com/PostView.naver?blogId=apricot_4&logNo=222658424853',
 'https://blog.naver.com/PostView.naver?blogId=skan5422&logNo=222583222026',
 'https://blog.naver.com/PostView.naver?blogId=semin1020&logNo=222661865508',
 'https://blog.naver.com/PostView.naver?blogId=rladnjsxo64&logNo=222565484664',
 'https://blog.naver.com/PostView.naver?blogId=ddihw&logNo=222452134060',
 'https://blog.naver.com/PostView.naver?blogId=pakseeun&logNo=222567999666',
 'https://blog.naver.com/PostView.naver?blogId=eter2000&logNo=222613

In [62]:
len(e)

149

In [43]:
result = []
for i in tqdm(range(len(e))):
    result.append(blog_content_parsing(a[i]))

100%|██████████████████████████████████████████████████████████████████████████████████| 96/96 [02:14<00:00,  1.40s/it]


In [17]:
df = pd.DataFrame()

In [18]:
df["content"] = result

In [13]:
len(result)

10

In [45]:
df = pd.DataFrame()
df["url"] = a
df["writer"] = b
df["title"] = d
df["description"] = e
df["content"] = result


In [63]:
df["협찬"] = 0

In [64]:
df["체험단"] = 0

In [65]:
df["협찬"][df["content"].str.contains("협찬")] = 1

C:\Users\User\AppData\Local\Temp\ipykernel_10564\1051424646.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["협찬"][df["content"].str.contains("협찬")] = 1


In [68]:
df["체험단"][df["content"].str.contains("체험단")] = 1

C:\Users\User\AppData\Local\Temp\ipykernel_10564\3795517829.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["체험단"][df["content"].str.contains("체험단")] = 1


In [69]:
df[df["content"].str.contains("체험단")]

,url,writer,title,description,content,협찬,체험단
11,https://blog.naver.com/PostView.naver?blogId=p...,pakseeun,"소니 WF-1000xm4, 2주간의 실 사용후기",고민 중인 분들에게 도움이 되길 바랍니다. ㅋㅋ 산책하기 좋은 가을 #노캔이어폰 #...,"안녕하세요. 세은파파입니다. 소니 1000 4 체험단에 참여하게 되어, 매주 100...",0,1
24,https://blog.naver.com/PostView.naver?blogId=p...,pakseeun,"소니 WF-1000XM4, 네 번째 이야기: 영화 감상과 케이스 후기",헤드폰에 도전하는 소니의 노이즈 캔슬링 이어폰 WF-1000xm4 체험단으로 전해드...,안녕하세요. 세은파파입니다. 헤드폰에 도전하는 소니의 노이즈 캔슬링 이어폰 1000...,0,1
53,https://blog.naver.com/PostView.naver?blogId=w...,wowyuns,소니 WF-1000XM4 후기 남편에게 몰입의즐거움을 선물하다,외출할때마다 휴대폰 말고 꼭 챙겨나가는 나의 애정템 소니 WF1000XM4 무선이어...,외출할때마다 휴대폰 말고 꼭 챙겨나가는 나의 애정템 소니 1000 4 무선이어폰 오...,0,1
77,https://blog.naver.com/PostView.naver?blogId=w...,wowyuns,소니 무선이어폰 WF1000XM4 개봉 후기 미친 성능,앞두고 몇일전 화상 줌으로 비대면 온라인 발대식도 했었어요 발대식때 들었던 설명대로...,몇달전 소니에서 발표한 무선 블루투스 이어폰 헤드폰에도전하다 1000 4 출시 소식...,0,1
81,https://blog.naver.com/PostView.naver?blogId=j...,jin8624,소니 WF-1000XM4 블루투스 이어폰 6주간 실사용 후기,"소니 WF-1000XM4 의 다양한 편의기능 - 노이즈캔슬링, Speak to Ch...",안녕하세요? 이솔파파입니다. 오늘은 소니 1000 4 블루투스 이어폰의 여섯번째 리...,0,1
82,https://blog.naver.com/PostView.naver?blogId=h...,hyy2831,헤드폰에 도전하는 노이즈 캔슬링 소니 WF-1000XM4 몰입 후기,(그 후기도 다음주에 기대하세요♡) SONY 소니에서 지난 6월 출시한 신제품 WF...,가을바람이 살랑살랑 불어오는 요즘 단풍물이 조금씩 물들고 있어서 야외활동하기 딱 좋...,1,1
84,https://blog.naver.com/PostView.naver?blogId=h...,hyy2831,무선이어폰 소니 노이즈캔슬링 WF-1000XM4 한달사용후기,그 후기로 또 찾아뵐게요~♡ #무선이어폰 #노이즈캔슬링이어폰 #소니 #소니이어폰 #...,어느덧 202년 마지막달 12월이 다가왔어요 올한해는 전보다 더 빠르게 흘러간것같아...,1,1


In [56]:
df.loc[0, "content"].contains("소니")

AttributeError: 'str' object has no attribute 'contains'

In [49]:
df["content"]

0     지난 몇 주간 소니 1000 4에 대한 사용 후기를 남겼는데 개봉기를 빼놓을 수 없...
1     노이즈 캔슬링 이어폰의 왕의 귀환! 소니 1000 4가 발매가 되었습니다. 전작인 ...
2     드디어 나왔습니다! 크!! 기다리고 기다렸던 소니 1000 4 노이즈캔슬링 이어폰입...
3     소니에서 이번에 새로 정식 출시한 1000 4는 전작 1000 3에서 확연하게 달라...
4     안녕하세요, 밝은불꽃입니다! 요즘 날씨가 더워서 헤드폰 대신 블루투스 이어폰을 들고...
                            ...                        
91    중고 헤드폰 매입후기, 뱅앤올룹슨 안녕하세요. 판매 하시는 장비 및 전자기기를 빠르...
92    중고 헤드폰 매입후기, 애플 에어팟 맥스 무선 헤드폰 안녕하세요. 판매 하시는 장비...
93    최근 들어 가성비 좋은 제품으로 핫하게 떠오르고 있는 소니 블루투스 노이즈 캔슬링 ...
94    중고 무선 이어폰 매입후기, 애플 에어팟프로 안녕하세요. 판매 하시는 장비 및 전자...
95    무선 헤드폰 매입, 스컬캔디 블루투스 헤드폰 매입 후기 안녕하세요. 판매 하시는 장...
Name: content, Length: 96, dtype: object

In [74]:
df["coupang"] = 0

In [77]:
df["coupang"][df["content"].str.contains("coupa")] = 1

C:\Users\User\AppData\Local\Temp\ipykernel_10564\285214680.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["coupang"][df["content"].str.contains("coupa")] = 1


In [78]:
df[df["coupang"]==1]

,url,writer,title,description,content,협찬,체험단,coupang


In [80]:
preprocess_sentence_kr("안녕하세요 sdf 다링비")

'안녕하세요 다링비'

In [104]:
def cleanhtml(raw_html):
    cleantext = re.sub('&nbsp;| |\t|\r|\n', '', raw_html)
#     cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [101]:
def cleanhtml(raw_html):

    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});&nbsp;| |\t|\r|\n')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [105]:
cleanhtml("&nbsp coupa.ng")

'&nbspcoupa.ng'

In [72]:
df.to_csv("소닉_wh-1000xm4.csv", index= False,encoding="utf-8-sig")